In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils.loader as data
import utils.col_refs as REF

In [20]:
# cost per usage by benefit 
for benefitID in data.BENEFITS[REF.benefitID]: 
    cost = data.EMPLOYEES_BENEFITS_USAGE.groupby(REF.benefitID)[REF.benefit_cost].sum()
    usage = data.EMPLOYEES_BENEFITS_USAGE.groupby(REF.benefitID)[REF.usage_freq].sum()
    cost_usage = cost / usage
print(cost_usage)

BenefitID
1     256.997095
2     217.031671
3     100.222606
4      75.198794
5      22.537208
6     176.362496
7     134.239704
8      38.109756
9     250.184729
10    282.115881
11    282.333101
12    246.734933
13     20.134804
14     22.959145
15    108.554684
16     25.413873
17    149.897059
18    219.271235
19    106.064157
20    142.715471
21     49.396182
22    177.388152
23    285.616882
24    235.683502
25    219.991228
26    150.303280
27    276.695981
28     50.014726
29    141.856736
30    212.524877
dtype: float64
